In [15]:
import os
import sys
import pandas as pd
import argparse
from datetime import datetime
import pathlib

# # Setup Django environment
# # Alternative 1: Use absolute path construction
# current_dir = os.path.abspath('')
# parent_dir = os.path.dirname(current_dir)
# sys.path.insert(0, parent_dir)

# # Alternative 2: Use pathlib for more modern path handling
# parent_path = pathlib.Path().absolute().parent
# sys.path.insert(0, str(parent_path))

# 新增：將上一層目錄加入 sys.path
parent_path = pathlib.Path().absolute().parent
sys.path.insert(0, str(parent_path))

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'website_configs.settings')
import django
django.setup()
# 重要：設定環境變數以允許在 Jupyter 的異步環境中執行同步操作
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Now we can import Django models
from app_user_keyword_db.models import NewsData

In [16]:
# Read CSV file
csv_file_path = '../app_user_keyword/dataset/yahoo_finace_preprocessed_200.csv'
# csv_file_path = '../app_user_keyword/dataset/cna_news_preprocessed_12weeks.csv'
df = pd.read_csv(csv_file_path, sep='|')
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link,sentiment2
0,yahoo_20250412_1,2025-04-12,財經特派,00878、0056「高股息ETF四天王」搶翻！1周合吸10萬人 00929獨退燒,台股本周史詩級震盪！史上最大跌點與最大漲點竟同週上演，連帶點燃高股息ETF買氣，「五大天王」...,0.67,暫無,"[('股息', 6), ('人數', 4), ('天王', 3), ('零股族', 2), ...","['台股', '本', '周', '史詩級', '震盪', '！', '史', '上', '...","['台股', '史詩級', '漲點', '股息', '天王', '國泰', '股息', '元...","[NerToken(word='五大天王', ner='ORG', idx=(41, 45)...","[('台股', 'Na'), ('本', 'Nes'), ('周', 'Nd'), ('史詩...",https://tw.stock.yahoo.com/news/00878%E3%80%81...,https://s.yimg.com/uu/api/res/1.2/zAZGq5bca1ul...,0.67


In [17]:
for idx, row in df.iterrows():
    try:
        date_obj = datetime.strptime(row['date'], '%Y-%m-%d').date()
        news_data, created = NewsData.objects.update_or_create(
            item_id=row['item_id'],
            defaults={
                'date': date_obj,
                'category': row['category'],
                'title': row['title'],
                'content': row['content'],
                'sentiment': row['sentiment'],
                'top_key_freq': row['top_key_freq'],
                'tokens': row['tokens'],
                'tokens_v2': row['tokens_v2'],
                'entities': row['entities'],
                'token_pos': row['token_pos'],
                'link': row['link'],
                'photo_link': row['photo_link'] if row['photo_link'] != "" and not pd.isna(row['photo_link']) else None,
            }
        )
        if created:
            created_count += 1
        else:
            updated_count += 1
    except Exception as e:
        print(f"Error at row {idx}: {e}")
        print(row)

print(f"✅ Created {created_count} new records.")
print(f"🔄 Updated {updated_count} existing records.")    

Error at row 0: 
item_id                                          yahoo_20250412_1
date                                                   2025-04-12
category                                                     財經特派
title                   00878、0056「高股息ETF四天王」搶翻！1周合吸10萬人 00929獨退燒
content         台股本周史詩級震盪！史上最大跌點與最大漲點竟同週上演，連帶點燃高股息ETF買氣，「五大天王」...
sentiment                                                    0.67
summary                                                        暫無
top_key_freq    [('股息', 6), ('人數', 4), ('天王', 3), ('零股族', 2), ...
tokens          ['台股', '本', '周', '史詩級', '震盪', '！', '史', '上', '...
tokens_v2       ['台股', '史詩級', '漲點', '股息', '天王', '國泰', '股息', '元...
entities        [NerToken(word='五大天王', ner='ORG', idx=(41, 45)...
token_pos       [('台股', 'Na'), ('本', 'Nes'), ('周', 'Nd'), ('史詩...
link            https://tw.stock.yahoo.com/news/00878%E3%80%81...
photo_link      https://s.yimg.com/uu/api/res/1.2/zAZGq5bca1ul...
sentiment2                                                 

KeyboardInterrupt: 

In [ ]:
created

NameError: name 'created' is not defined